# Process the Date and Time of the Annual Hottest and Coldest Hour by Balancing Authority


In [1]:
# Start by importing the packages we need:
import os

import pandas as pd
import numpy as np

from glob import glob
from datetime import timedelta


## Set the Directory Structure

In [2]:
# Identify the data input and output directories:
data_input_dir =  '/Users/burl878/Documents/IMMM/Data/TELL/Production_Runs/tell_data/wrf_to_tell_data/historic/'
metadata_input_dir =  '/Users/burl878/Documents/IMMM/Data/TELL/Analysis/'
data_output_dir =  '/Users/burl878/Documents/IMMM/Data/TELL/Analysis/Hottest_Coldest_Days/'


## Process the Data

In [40]:
# Define a function to process the daily maximum and minimum temperature time series:
def process_hottest_coldest_days_time_series(data_input_dir: str, metadata_input_dir: str, data_output_dir: str, interconnection_to_process: str):
    
    # Read in the BA-to-Interconnection mapping file:
    ba_mapping = pd.read_csv(metadata_input_dir + 'BA_to_Interconnection_Mapping.csv')
    
    # Subset to just the BAs for the interconnection being processed:
    ba_mapping = ba_mapping.loc[ba_mapping['Interconnection'] == interconnection_to_process]
    
    # Make a list of all of the BAs in "ba_mapping":
    bas = ba_mapping['BA_Code'].unique()
    
    #Initiate a counter to store the results:
    counter = 0;
    output_df = pd.DataFrame()
    
    # Loop over the BAs and find the hottest and coldest day for each year:
    for i in range(len(bas)):
        # Loop over the years from the 1980 to 2019:
        for year in range(1980,2020,1):
            # Iterate the counter by one:
            counter = counter + 1
            
            # Create the filename for the given BA and year combination:
            filename = (data_input_dir + bas[i] + '_WRF_Hourly_Mean_Meteorology_' + str(year) + '.csv')
    
            # Read in the .csv file:
            met_df = pd.read_csv(filename)
    
            # Set the time variable as an index:
            met_df.index = pd.to_datetime(met_df['Time_UTC'])
        
            # Convert the temperature from Kelvin to Fahrenheit:
            met_df['T2'] = (1.8 * (met_df['T2'] - 273)) + 32
    
            # Add the hottest and coldest times to the output file:
            output_df.loc[counter, 'BA'] = bas[i]
            output_df.loc[counter, 'Year'] = str(year)
            output_df.loc[counter, 'Coldest_Hour_UTC'] = met_df['Time_UTC'].loc[met_df['T2'].idxmin()]
            output_df.loc[counter, 'Coldest_Temp_F'] = met_df['T2'].min().round(2)
            output_df.loc[counter, 'Hottest_Hour_UTC'] = met_df['Time_UTC'].loc[met_df['T2'].idxmax()]
            output_df.loc[counter, 'Hottest_Temp_F'] = met_df['T2'].max().round(2)
        
            # Clean up and move to the next year:
            del filename, met_df
        
    # Write out the dataframe to a .csv file:
    output_df.to_csv((os.path.join(data_output_dir + interconnection_to_process + '_Hottest_Coldest_Days_1980_to_2019.csv')), sep=',', index=False)
    
    return output_df


In [43]:
output_df = process_hottest_coldest_days_time_series(data_input_dir = data_input_dir, 
                                                     metadata_input_dir = metadata_input_dir,
                                                     data_output_dir = data_output_dir, 
                                                     interconnection_to_process = 'WECC')

output_df


,BA,Year,Coldest_Hour_UTC,Coldest_Temp_F,Hottest_Hour_UTC,Hottest_Temp_F
1,AVA,1980,1980-01-29 15:00:00,-10.19,1980-07-22 23:00:00,96.76
2,AVA,1981,1981-02-10 15:00:00,-1.98,1981-08-13 23:00:00,95.52
3,AVA,1982,1982-01-06 15:00:00,-15.54,1982-08-07 23:00:00,93.36
4,AVA,1983,1983-12-23 16:00:00,-15.07,1983-08-07 23:00:00,95.38
5,AVA,1984,1984-12-19 06:00:00,-3.28,1984-07-25 23:00:00,96.15
...,...,...,...,...,...,...
1116,WAUW,2015,2015-01-09 07:00:00,-8.05,2015-08-14 20:00:00,96.21
1117,WAUW,2016,2016-12-17 13:00:00,-19.32,2016-07-22 21:00:00,95.22
1118,WAUW,2017,2017-12-31 10:00:00,-16.42,2017-07-08 22:00:00,95.18
1119,WAUW,2018,2018-02-20 14:00:00,-16.15,2018-08-11 21:00:00,99.19
